# 資料載入與初步檢視


初步載入歷史 Tick/分K 資料並進行初步檢視。


In [55]:
# 載入 pandas 函式庫
import pandas as pd

# TODO: 指定資料檔案路徑
data_path = '../data/ticks_2025-04.parquet'

# TODO: 載入資料
try:
    df = pd.read_parquet(data_path)
    print("資料載入成功！")
    print("前 5 行資料：")
    print(df.head())
    print("\n資料基本資訊：")
    df.info()
    print("\n描述性統計：")
    print(df.describe())
except FileNotFoundError:
    print(f"錯誤：找不到檔案 {data_path}")
except Exception as e:
    print(f"載入資料時發生錯誤：{e}")


資料載入成功！
前 5 行資料：
             ts    close  volume  bid_price  bid_volume  ask_price  \
0  1.743433e+18  20807.0    89.0        0.0         0.0        0.0   
1  1.743433e+18  20808.0     2.0    20807.0         5.0    20808.0   
3  1.743433e+18  20809.0     1.0    20807.0         5.0    20808.0   
5  1.743433e+18  20807.0     1.0    20807.0         5.0    20810.0   
6  1.743433e+18  20807.0     1.0    20807.0         6.0    20810.0   

   ask_volume  tick_type               timestamp  is_traffic_limited  
0         0.0        1.0 2025-03-31 15:00:00.021               False  
1         2.0        1.0 2025-03-31 15:00:00.045               False  
3         2.0        1.0 2025-03-31 15:00:00.051               False  
5        15.0        2.0 2025-03-31 15:00:00.118               False  
6        18.0        2.0 2025-03-31 15:00:00.176               False  

資料基本資訊：
<class 'pandas.core.frame.DataFrame'>
Index: 3623517 entries, 0 to 126096
Data columns (total 10 columns):
 #   Column         

In [56]:
df.columns
len(df)

3623517

# 資料清洗


處理資料中的缺失值、異常值和重複資料。


In [57]:
# TODO: 處理缺失值 (NaN)
print("\n處理缺失值前：")
print(df.isnull().sum())
df = df.dropna() # 範例：簡單刪除含有缺失值的列
print("\n處理缺失值後：")
print(df.isnull().sum())

# TODO: 處理異常價格 (outliers)
# 根據實際資料情況定義異常值判斷邏輯，例如基於標準差或 IQR
print("\n處理異常值前：")
print(df.describe())
# 範例：移除價格為 0 或極端值的資料
df = df[(df['close'] > 0) & (df['close'] < df['close'].quantile(0.99))]
print("\n處理異常值後：")
print(df.describe())

# TODO: 處理重複資料
print("\n處理重複資料前：")
print(f"重複資料筆數：{df.duplicated().sum()}")
df = df.drop_duplicates(subset=['timestamp'])
print("\n處理重複資料後：")
print(f"重複資料筆數：{df.duplicated().sum()}")



處理缺失值前：
ts                    4
close                 4
volume                4
bid_price             4
bid_volume            4
ask_price             4
ask_volume            4
tick_type             4
timestamp             0
is_traffic_limited    0
dtype: int64

處理缺失值後：
ts                    0
close                 0
volume                0
bid_price             0
bid_volume            0
ask_price             0
ask_volume            0
tick_type             0
timestamp             0
is_traffic_limited    0
dtype: int64

處理異常值前：
                 ts         close        volume     bid_price    bid_volume  \
count  3.623513e+06  3.623513e+06  3.623513e+06  3.623513e+06  3.623513e+06   
mean   1.744626e+18  1.926879e+04  1.497686e+00  1.925888e+04  6.805559e+00   
min    1.743433e+18  1.701500e+04  1.000000e+00  0.000000e+00  0.000000e+00   
25%    1.744153e+18  1.863300e+04  1.000000e+00  1.863000e+04  2.000000e+00   
50%    1.744401e+18  1.934800e+04  1.000000e+00  1.934700e+04  3.000000e

In [58]:
print("Timestamp 範圍：")
print(f"最早時間: {df['timestamp'].min()}")
print(f"最晚時間: {df['timestamp'].max()}")
   

Timestamp 範圍：
最早時間: 2025-03-31 15:00:00.021000
最晚時間: 2025-04-30 13:44:59.770000


# 時間序列處理


將時間欄位設定為索引，標準化時間格式，並處理非交易時間資料。


In [59]:
# TODO: 將時間欄位設定為索引
# 假設時間欄位名稱為 'timestamp'
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# TODO: 標準化時間格式 (如果需要)
# 確保時間索引的頻率正確，例如 'T' for minute, 'S' for second
df = df.sort_index()

# TODO: 處理非交易時間資料 (如果需要)
# 根據實際交易時間定義，移除或處理非交易時段的資料
# 例如：只保留交易時間內的資料
# 定義交易時間範圍 (日盤和夜盤)
day_start = '08:45'
day_end = '13:45'
night_start = '15:00'
night_end = '05:00'

# 篩選日盤和夜盤資料
day_session = df.between_time(day_start, day_end)
night_session = df.between_time(night_start, night_end)

# 合併日盤和夜盤資料並重新排序
df = pd.concat([day_session, night_session]).sort_index()

print("\n時間序列處理後：")
print(df.head())
print(df.index.dtype)



時間序列處理後：
                                   ts    close  volume  bid_price  bid_volume  \
timestamp                                                                       
2025-03-31 15:00:00.021  1.743433e+18  20807.0    89.0        0.0         0.0   
2025-03-31 15:00:00.045  1.743433e+18  20808.0     2.0    20807.0         5.0   
2025-03-31 15:00:00.051  1.743433e+18  20809.0     1.0    20807.0         5.0   
2025-03-31 15:00:00.118  1.743433e+18  20807.0     1.0    20807.0         5.0   
2025-03-31 15:00:00.176  1.743433e+18  20807.0     1.0    20807.0         6.0   

                         ask_price  ask_volume  tick_type  is_traffic_limited  
timestamp                                                                      
2025-03-31 15:00:00.021        0.0         0.0        1.0               False  
2025-03-31 15:00:00.045    20808.0         2.0        1.0               False  
2025-03-31 15:00:00.051    20808.0         2.0        1.0               False  
2025-03-31 15:00:00.11

In [65]:
len(df)
# df.head()

3587183

# 初步探索性資料分析 (EDA)


進行初步的探索性資料分析，可視化價格、成交量等數據。


In [ ]:
import matplotlib.pyplot as plt
# import seaborn as sns # 如果需要使用 seaborn

# TODO: 繪製價格 (開高低收) 走勢圖
# plt.figure(figsize=(12, 6))
# plt.plot(df.index, df['close'], label='收盤價')
# plt.title('價格走勢圖')
# plt.xlabel('時間')
# plt.ylabel('價格')
# plt.legend()
# plt.show()

# TODO: 繪製成交量變化圖
# plt.figure(figsize=(12, 6))
# plt.bar(df.index, df['volume'], label='成交量')
# plt.title('成交量變化圖')
# plt.xlabel('時間')
# plt.ylabel('成交量')
# plt.legend()
# plt.show()

# TODO: 基礎波動率分析 (e.g., 日報酬率標準差)
# 如果資料是 Tick 或分K，可能需要先 resampling 到日頻率
# daily_returns = df['close'].resample('D').ffill().pct_change().dropna()
# daily_volatility = daily_returns.std()
# print(f"\n日報酬率標準差 (波動率): {daily_volatility:.4f}")

# TODO: 價格與成交量分佈圖
# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# sns.histplot(df['close'], kde=True)
# plt.title('價格分佈')
# plt.xlabel('價格')
# plt.ylabel('頻率')

# plt.subplot(1, 2, 2)
# sns.histplot(df['volume'], kde=True)
# plt.title('成交量分佈')
# plt.xlabel('成交量')
# plt.ylabel('頻率')
# plt.tight_layout()
# plt.show()


# 技術指標特徵工程 (基礎)


計算基礎的技術指標作為初步特徵。


In [ ]:
# TODO: 計算基礎技術指標 (e.g., SMA, EMA, RSI)
# 需要安裝 ta-lib 或使用 pandas 內建函數
# 例如使用 pandas rolling 計算 SMA
# df['SMA_10'] = df['close'].rolling(window=10).mean()

# 例如使用 ta-lib (需要先安裝)
# import talib
# df['RSI_14'] = talib.RSI(df['close'], timeperiod=14)

# TODO: 考慮將特徵工程函數移至 features/feature_engineering.py
# from features.feature_engineering import calculate_sma
# df['SMA_20'] = calculate_sma(df['close'], window=20)

# print("\n新增技術指標特徵後：")
# print(df.head())


# 製作標註欄位 (Target Labeling)


定義預測目標並創建對應的標註欄位。


In [ ]:
# TODO: 定義預測目標 (e.g., 未來 N Tick/分鐘 後價格漲跌幅是否超過 X%)
# 範例：預測未來 5 分鐘收盤價是否上漲超過 0.1%
# lookahead_window = '5T' # 5 分鐘
# price_change_threshold = 0.001 # 0.1%

# TODO: 創建標註欄位
# future_price = df['close'].shift(-pd.Timedelta(lookahead_window))
# price_change = (future_price - df['close']) / df['close']
# df['target_price_up'] = (price_change > price_change_threshold).astype(int)

# TODO: 處理標註欄位中的 NaN (由於 shift 操作會產生 NaN)
# df.dropna(subset=['target_price_up'], inplace=True)

# TODO: 考慮將標註函數移至 features/labeling.py
# from features.labeling import create_up_down_label
# df['target_up_down'] = create_up_down_label(df['close'], window='10T', threshold=0.002)

# print("\n新增標註欄位後：")
# print(df.head())
# print("\n標註欄位分佈：")
# print(df['target_price_up'].value_counts())
